### Imports and General Data

In [1]:
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from preprocessing import create_connection, get_files
from main import get_files, get_data, export_data
(root, data_path, presidents, president, cities, years) = get_data()

# Aggregated Daily Sentiment

In [ ]:
trump = pd.read_pickle(f'{data_path}trump.pkl')
johnson = pd.read_pickle(f'{data_path}johnson.pkl')

In [ ]:
trump.shape, johnson.shape

((0, 0), (0, 0))

In [ ]:
trump.columns

In [ ]:
trump.drop(columns=[])

In [ ]:
# trump[['created_at', 'created_timestamp']]
trump[['created_timestamp']]
# format: UTC time ()

In [ ]:
# TODO: SQL STATEMENT GORUP BY DAY?